In [1]:
# Importing required liberaries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df['label'].unique()

array([1, 0])

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.dropna(inplace = True)


In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [8]:
## Dividing data in dependent and independent features .

X = df.drop('label', axis = 1)
y = df['label']

In [9]:
df.shape

(18285, 5)

In [10]:
print(tf.__version__)

2.15.0


In [11]:
## Importing important tools for model

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [12]:
## Setting up vocabulary size

voc_size = 5000


In [13]:
## Creating Embedding layer
massages = X.copy()
massages.reset_index(inplace = True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
## Text preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(massages)):
  review = re.sub('[^a-zA-Z]',' ', massages['title'][i])
  review = review.lower()
  review = review.split()

  ## Stemming and removing stopwords
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
## Performing one hot encoding

onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[1]

[1101, 4604, 4923, 982, 1982, 2625, 949]

In [19]:
## Creating Embedding layer
## padding to make length of all sentences same

sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1535 1281 2179]
 [   0    0    0 ... 1982 2625  949]
 [   0    0    0 ... 4289 3675  414]
 ...
 [   0    0    0 ... 2727  924 1622]
 [   0    0    0 ... 1710 4536 3115]
 [   0    0    0 ... 2674 2831 3266]]


In [20]:
## Creating a model

embedding_vector_features = 40  # Dimensions
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
### Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 20,min_delta = 0.0001, verbose = 1,mode = 'auto', baseline = None, restore_best_weights = False )

In [22]:
## Final data

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [23]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [24]:
## Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.20, random_state = 42)

In [25]:
## Model training

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 30, batch_size = 64, callbacks = early_stopping)

Epoch 1/30
229/229 [==============================] - 14s 50ms/step - loss: 0.3209 - accuracy: 0.8536 - val_loss: 0.2024 - val_accuracy: 0.9103
Epoch 2/30
229/229 [==============================] - 9s 38ms/step - loss: 0.1359 - accuracy: 0.9474 - val_loss: 0.2002 - val_accuracy: 0.9147
Epoch 3/30
229/229 [==============================] - 11s 48ms/step - loss: 0.0970 - accuracy: 0.9636 - val_loss: 0.2373 - val_accuracy: 0.9005
Epoch 4/30
229/229 [==============================] - 13s 55ms/step - loss: 0.0683 - accuracy: 0.9757 - val_loss: 0.2508 - val_accuracy: 0.9122
Epoch 5/30
229/229 [==============================] - 8s 36ms/step - loss: 0.0449 - accuracy: 0.9851 - val_loss: 0.3105 - val_accuracy: 0.9155
Epoch 6/30
229/229 [==============================] - 11s 48ms/step - loss: 0.0318 - accuracy: 0.9893 - val_loss: 0.4125 - val_accuracy: 0.9065
Epoch 7/30
229/229 [==============================] - 11s 46ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.4412 - val_accuracy: 0

In [30]:
### Performance metrics

y_pred = model.predict(X_test)

115/115 [==============================] - 1s 7ms/step


In [31]:
y_pred = np.where(y_pred >0.5, 1, 0)

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1906  176]
 [ 165 1410]]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      2082
           1       0.89      0.90      0.89      1575

    accuracy                           0.91      3657
   macro avg       0.90      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657



In [33]:
## Accuracy score

from sklearn.metrics import accuracy_score

score =accuracy_score(y_test, y_pred)
print(score)

0.9067541700847689
